### Creation of traditonal event log out of triplges generated by trackintel package. 
#### Analysis of event log using pm4py

#### 1. Import libraries

In [87]:
import sys
import os
import pm4py
# Assuming you're in notebooks/, go one level up
sys.path.append(os.path.abspath(".."))

from pm4mobility.io.loader import *
from pm4mobility.analysis import flows
from pm4mobility.log import event_log
#from mobility4py.log.event_log import create_event_log


#### 2. Read triplges from geojson file

In [88]:
df = load_triplegs_geojson("../data/triplegs.geojson")

In [89]:
df.head(10)

,id,trip_id,started_at,finished_at,length,speed,duration_minutes,mode,geometry
0,13,0,2024-02-02 08:47:14+00:00,2024-02-02 08:47:42+00:00,325.714568,11.632663,0.466667,motorized_mobility,"LINESTRING (-9.15809 38.71762, -9.16165 38.71669)"
1,14,0,2024-02-02 09:11:50+00:00,2024-02-02 09:12:27+00:00,325.714568,8.803096,0.616667,motorized_mobility,"LINESTRING (-9.16165 38.71669, -9.15809 38.71762)"
2,19,1,2024-02-03 12:05:23+00:00,2024-02-03 12:08:59+00:00,325.714568,1.507938,3.600000,slow_mobility,"LINESTRING (-9.15809 38.71762, -9.15809 38.717..."
3,22,2,2024-02-05 20:06:19+00:00,2024-02-05 20:10:53+00:00,325.714568,1.188739,4.566667,slow_mobility,"LINESTRING (-9.16165 38.71669, -9.15809 38.71762)"
4,24,3,2024-02-06 11:26:38+00:00,2024-02-06 11:30:12+00:00,325.714568,1.522031,3.566667,slow_mobility,"LINESTRING (-9.16165 38.71669, -9.15809 38.71762)"
5,37,4,2024-02-13 12:31:44+00:00,2024-02-13 12:45:48+00:00,325.714568,0.385918,14.066667,slow_mobility,"LINESTRING (-9.16165 38.71669, -9.15809 38.717..."
6,38,5,2024-02-14 11:12:05+00:00,2024-02-14 11:17:00+00:00,325.714568,1.104117,4.916667,slow_mobility,"LINESTRING (-9.15809 38.71762, -9.16165 38.71669)"
7,42,6,2024-02-16 07:29:43+00:00,2024-02-16 07:33:22+00:00,325.714568,1.487281,3.650000,slow_mobility,"LINESTRING (-9.15809 38.71762, -9.16165 38.71669)"
8,54,7,2024-02-25 12:19:13+00:00,2024-02-25 12:21:27+00:00,325.714568,2.430706,2.233333,slow_mobility,"LINESTRING (-9.16165 38.71669, -9.16165 38.716..."
9,55,8,2024-02-25 20:15:11+00:00,2024-02-25 20:16:21+00:00,325.714568,4.653065,1.166667,motorized_mobility,"LINESTRING (-9.15809 38.71762, -9.16165 38.71669)"


#### 3. Read districts file 

In [90]:
districts = load_districts_layer("../data/cont-aad-caop2017")

In [91]:
districts.head(10)

,Dicofre,Freguesia,Concelho,Distrito,TAA,AREA_EA_Ha,AREA_T_Ha,Des_Simpli,geometry
0,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.09,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.28204 37.07355, -8.28205 37.07354..."
1,081504,Sagres,VILA DO BISPO,FARO,ÁREA PRINCIPAL,3431.83,3436.91,Sagres,"POLYGON ((-8.89529 37.04182, -8.89533 37.04175..."
2,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30727 37.07449, -8.30727 37.07449..."
3,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30764 37.07458, -8.30766 37.07458..."
4,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30777 37.07466, -8.30778 37.07466..."
5,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30741 37.07488, -8.30742 37.07487..."
6,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30824 37.07497, -8.30825 37.07497..."
7,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.27701 37.07564, -8.27702 37.07564..."
8,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.02,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.27663 37.07574, -8.27664 37.07574..."
9,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.27658 37.07594, -8.27658 37.07594..."


#### 4. Create Origin-Destination Flows out of triplges

In [92]:
od_flows = flows.create_od_flows(df, districts)

In [93]:
od_flows.head(10)

,trip_id,origin,started_at,destination,finished_at,origin_area,dest_area,origin_antenna,dest_antenna,origin_mode,dest_mode,duration_minutes,speed,origin_length,dest_length
18,16,POINT (-9.18948 38.77278),2024-02-05 20:35:19+00:00,POINT (-9.19849 38.76623),2024-02-05 20:41:48+00:00,LISBOA,ODIVELAS,Carnide,Pontinha e Famões,slow_mobility,slow_mobility,6.483333,2.745047,1067.823472,1067.823472
24,22,POINT (-9.18769 38.78616),2024-02-16 18:37:51+00:00,POINT (-9.18948 38.77278),2024-02-16 18:39:55+00:00,ODIVELAS,LISBOA,Odivelas,Carnide,motorized_mobility,motorized_mobility,2.066667,12.061921,1495.678151,1495.678151
27,25,POINT (-9.19206 38.79794),2024-02-27 19:05:57+00:00,POINT (-9.20114 38.80444),2024-02-27 19:10:39+00:00,ODIVELAS,ODIVELAS,Odivelas,Ramada e Caneças,slow_mobility,slow_mobility,4.700000,3.789262,1068.571802,1068.571802
34,32,POINT (-9.19206 38.79794),2024-03-12 18:09:11+00:00,POINT (-9.18744 38.80551),2024-03-12 18:11:11+00:00,ODIVELAS,ODIVELAS,Odivelas,Ramada e Caneças,motorized_mobility,motorized_mobility,2.000000,7.763965,931.675799,931.675799
35,33,POINT (-9.12486 38.77111),2024-02-04 19:52:29+00:00,POINT (-9.1361 38.75947),2024-02-04 19:53:23+00:00,LISBOA,LISBOA,Olivais,Alvalade,fast_mobility,fast_mobility,0.900000,30.014186,1620.766057,1620.766057
38,36,POINT (-9.14247 38.75119),2024-02-23 21:53:47+00:00,POINT (-9.12486 38.77111),2024-02-23 21:58:00+00:00,LISBOA,LISBOA,Alvalade,Olivais,motorized_mobility,motorized_mobility,4.216667,10.647495,2693.816128,2693.816128
39,37,POINT (-9.12486 38.77111),2024-02-25 17:07:58+00:00,POINT (-9.1361 38.75947),2024-02-25 17:10:14+00:00,LISBOA,LISBOA,Olivais,Alvalade,motorized_mobility,motorized_mobility,2.266667,11.917397,1620.766057,1620.766057
40,38,POINT (-9.12486 38.77111),2024-03-01 13:53:19+00:00,POINT (-9.1361 38.75947),2024-03-01 13:56:09+00:00,LISBOA,LISBOA,Olivais,Alvalade,motorized_mobility,motorized_mobility,2.833333,9.533918,1620.766057,1620.766057
42,39,POINT (-9.1361 38.75947),2024-03-05 19:46:06+00:00,POINT (-9.12486 38.77111),2024-03-05 19:48:24+00:00,LISBOA,LISBOA,Alvalade,Olivais,motorized_mobility,motorized_mobility,2.300000,11.744682,1620.766057,1620.766057
46,42,POINT (-9.19773 38.74461),2024-02-05 17:37:38+00:00,POINT (-9.20687 38.74136),2024-02-05 17:41:03+00:00,LISBOA,AMADORA,Benfica,Águas Livres,motorized_mobility,motorized_mobility,3.416667,4.250192,871.289349,871.289349


#### 5. Filtering flows only to municipality of Oeiras

In [94]:
oeiras = od_flows.loc[(od_flows['origin_area'] == 'OEIRAS') | (od_flows['dest_area'] == 'OEIRAS')]

#### 6. Create traditional event log out of flows

In [95]:
events = event_log.create_trad_event_log(oeiras)

In [96]:
events.head(10)

,caseid,timestamp,event,antenna,length,mode
0,55,2024-03-08 09:56:17+00:00,OEIRAS,Barcarena,1581.642329,motorized_mobility
54387,55,2024-03-08 09:58:42+00:00,SINTRA,Massamá e Monte Abraão,1581.642329,motorized_mobility
1,59,2024-03-13 20:18:23+00:00,OEIRAS,Barcarena,1193.163097,motorized_mobility
54388,59,2024-03-13 20:20:01+00:00,SINTRA,Massamá e Monte Abraão,1193.163097,motorized_mobility
2,60,2024-02-01 10:27:59+00:00,CASCAIS,São Domingos de Rana,4140.996906,motorized_mobility
54389,60,2024-02-01 10:32:12+00:00,OEIRAS,"Oeiras e S. Julião da Barra, Paço de Arcos e C...",4140.996906,motorized_mobility
3,61,2024-02-05 14:18:50+00:00,LISBOA,Estrela,7581.190023,motorized_mobility
54390,61,2024-02-05 14:25:25+00:00,OEIRAS,Carnaxide e Queijas,7581.190023,motorized_mobility
4,64,2024-02-13 19:06:14+00:00,OEIRAS,"Oeiras e S. Julião da Barra, Paço de Arcos e C...",2848.202671,motorized_mobility
54391,64,2024-02-13 19:08:10+00:00,CASCAIS,Carcavelos e Parede,2848.202671,motorized_mobility


#### 7. Load events into PM4PY dataframe, create two logs: municipality as event and antenna as event

In [97]:
events_df_municipality = pm4py.format_dataframe(events, case_id='caseid', activity_key='event', timestamp_key='timestamp')
log_municipality = pm4py.convert_to_event_log(events_df_municipality)

In [98]:
events_df_district = pm4py.format_dataframe(events, case_id='caseid', activity_key='antenna', timestamp_key='timestamp')
log_district = pm4py.convert_to_event_log(events_df_district)

#### 8. Discover process model using heuristic miner

In [72]:
heu_net_m = pm4py.discover_heuristics_net(log_municipality)
pm4py.save_vis_heuristics_net(heu_net_m, '../figures/oeiras_heu_net_dfg_municipality.png')

''

In [73]:
heu_net_d = pm4py.discover_heuristics_net(log_district)
pm4py.save_vis_heuristics_net(heu_net_d, '../figures/oeiras_heu_net_dfg_district.png')

''

#### 9. Discover Petri net

In [86]:
net, im, fm = pm4py.discover_petri_net_inductive(log_district)
net = pm4py.reduce_petri_net_invisibles(net)
pm4py.save_vis_petri_net(net, im, fm, '../figures/petri_net_districts.png')

''